In [ ]:
# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

In [1]:
import json
import os
import pandas as pd
import pyarrow
import numpy as np

import sys
from os import path
import numpy

from dotenv import load_dotenv
from sqlalchemy import create_engine


load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')

/Users/victoria/GitHub/venvs/bespoke/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
import create_queries
%load_ext autoreload
%autoreload 2

In [3]:
import prepare_data

In [4]:
pd.options.display.float_format = '{:,.2f}'.format

In [5]:
from datetime import date

In [6]:
today = date.today()
today

datetime.date(2022, 2, 12)

In [7]:
COMPANY_NAME = 'DL'
COMPANY_IDENTIFIER = 'DL'
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'

INVENTORY_DATE = '02/07/2022'

In [8]:
company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_sales_transactions_query = create_queries.create_company_sales_transactions_query(COMPANY_IDENTIFIER, SALES_TRANSACTIONS_START_DATE)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(COMPANY_IDENTIFIER)

engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))
company_incoming_transfer_packages_dataframe = pd.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pd.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_sales_transactions_dataframe = pd.read_sql_query(company_sales_transactions_query, engine)
company_inventory_packages_dataframe = pd.read_sql_query(company_inventory_packages_query, engine)

In [9]:
deduped_sales_receipts_dataframe = prepare_data.dedupe_sales_transactions(company_sales_transactions_dataframe)

In [10]:
df_in_og = company_incoming_transfer_packages_dataframe
df_inventory_og = company_inventory_packages_dataframe
df_sales_og = deduped_sales_receipts_dataframe

In [11]:
df_in = df_in_og
df_inventory = df_inventory_og
df_sales = df_sales_og

In [12]:
license_numbers = list(df_in['license_number'].unique())
license_numbers

['MR283369']

In [13]:
license_number = license_numbers[0]

In [14]:
df_license_specific = df_in[df_in['license_number'] == license_number]
df_license_specific.reset_index(drop=True, inplace=True)
legal_name = df_license_specific['recipient_facility_name'][0]

In [15]:
df_sales = df_sales[df_sales['license_number'] == license_number]
df_inventory = df_inventory[df_inventory['license_number'] == license_number]

In [ ]:
# df_in.groupby('recipient_facility_name').sum()
# df_sales_licenses = df_sales.groupby('license_number').sum()
# df_sales_licenses.to_csv('./TL_licenses.csv')
# df_inventory.groupby('license_number').sum()
# df_sales = df_sales[(df_sales['license_number'] == '402-00473') | (df_sales['license_number'] == '402R-00536')]
# df_inventory = df_inventory[(df_inventory['license_number'] == '402-00473') | (df_inventory['license_number'] == '402R-00536')]

## Sales

In [16]:
df_sales['per_unit'] = df_sales['tx_total_price'] / df_sales['tx_quantity_sold']
df_sales['year_month'] = df_sales['sales_datetime'].dt.strftime("%Y-%m")

In [17]:
s_revenue = df_sales.groupby('year_month')['tx_total_price'].sum()
df_revenue = pd.Series(s_revenue).to_frame()
df_revenue = df_revenue.reset_index()
df_revenue.rename(columns={'tx_total_price': 'revenue'}, inplace=True)

In [18]:
s_total_count = df_sales.groupby('year_month')['tx_total_price'].count()
df_total_count = pd.Series(s_total_count).to_frame()
df_total_count = df_total_count.reset_index()
df_total_count.rename(columns={'tx_total_price':'total_count'}, inplace=True)

In [ ]:
# df_sales.to_csv('./WTH_sales.csv')

## Vendor Churn

In [ ]:
df_in.to_csv('./verts_incoming.csv')

In [ ]:
df_vendor_churn = df_in[df_in['license_number'] == license_number]

In [ ]:
df_vendor_churn.to_csv('./' + license_number + '_incoming.csv')

In [ ]:
# df_sales.to_csv('./' + license_number + '_sales.csv')

In [ ]:
# df_vendor_churn = df_in[(df_in['license_number'] == '402-01142') | (df_in['license_number'] == '402R-00573')]

In [ ]:
# df_vendor_churn.to_csv('./Tweedleaf_Jason_incoming.csv')

## Incoming - get the avg price per package id then product name

In [ ]:
# df_in = df_in[df_in['shipper_wholesale_price'] > 1]

In [19]:
df_in['per_unit_incoming'] = df_in['shipper_wholesale_price'] / df_in['shipped_quantity']
df_in_price = df_in[df_in['shipper_wholesale_price'].notnull()]

In [20]:
average_incoming_package_id = df_in_price.groupby('package_id')['per_unit_incoming'].mean()
df_avg_incoming_price = pd.Series(average_incoming_package_id).to_frame()
df_avg_incoming_price = df_avg_incoming_price.reset_index()

In [21]:
average_incoming_product = df_in_price.groupby('product_name')['per_unit_incoming'].mean()
df_avg_product = pd.Series(average_incoming_product).to_frame()
df_avg_product = df_avg_product.reset_index()
df_avg_product.rename(columns={'per_unit_incoming':'per_unit_product'}, inplace=True)

## COGS

In [ ]:
df_cogs_package_id = pd.merge(df_sales, df_avg_incoming_price, left_on='tx_package_id', right_on='package_id', how='left')
df_cogs_package_id['total_incoming'] = df_cogs_package_id['per_unit_incoming'] * df_cogs_package_id['tx_quantity_sold']
df_cogs_package_id.replace([np.inf], np.nan, inplace=True)
df_cogs_package_id_notnull = df_cogs_package_id[df_cogs_package_id['total_incoming'].notnull()]

In [ ]:
s_cogs = df_cogs_package_id_notnull.groupby('year_month')['total_incoming'].sum()
df_cogs_id = pd.Series(s_cogs).to_frame()
df_cogs_id = df_cogs_id.reset_index()

In [ ]:
s_cogs_count = df_cogs_package_id_notnull.groupby('year_month')['total_incoming'].count()
df_cogs_count = pd.Series(s_cogs_count).to_frame()
df_cogs_count = df_cogs_count.reset_index()
df_cogs_count.rename(columns={'total_incoming':'count_incoming'}, inplace=True)

In [ ]:
df_cogs_average_product = pd.merge(df_cogs_package_id, df_avg_product, left_on='tx_product_name', right_on='product_name', how='left')
df_cogs_average_product['total_product'] = df_cogs_average_product['tx_quantity_sold'] * df_cogs_average_product['per_unit_product']
df_cogs_null = df_cogs_average_product[df_cogs_average_product['per_unit_incoming'].isnull()]
df_cogs_product = df_cogs_null[df_cogs_null['per_unit_product'].notnull()]

In [ ]:
product_sum = df_cogs_product.groupby('year_month')['total_product'].sum()
df_product_sum = pd.Series(product_sum).to_frame()
df_product_sum = df_product_sum.reset_index()
df_product_sum.rename(columns={'total_product':'product_sum'}, inplace=True)

In [ ]:
product_count = df_cogs_product.groupby('year_month')['total_product'].count()
df_product_count = pd.Series(product_count).to_frame()
df_product_count = df_product_count.reset_index()
df_product_count.rename(columns={'total_product':'product_count'}, inplace=True)

In [ ]:
df_cogs_product = pd.merge(df_product_sum, df_product_count)

In [ ]:
# df_cogs_package_id.to_csv('./UHHC_cogs.csv')

## Summary

In [ ]:
df_summary = pd.merge(df_revenue, df_cogs_product, how='left')
df_summary = pd.merge(df_summary, df_cogs_id, how='left')
df_summary['product_sum'] = df_summary['product_sum'].fillna(0)
df_summary['product_count'] = df_summary['product_count'].fillna(0)
df_summary['cogs'] = df_summary['total_incoming'] + df_summary['product_sum']
df_summary = pd.merge(df_summary, df_cogs_count)
df_summary = pd.merge(df_summary, df_total_count)
df_summary['total_count_incoming'] = df_summary['product_count'] + df_summary['count_incoming']
df_summary['margin_$'] = df_summary['revenue'] - df_summary['cogs']
df_summary['margin_%'] = df_summary['margin_$'] / df_summary['revenue']
df_summary['coverage'] = df_summary['total_count_incoming'] / df_summary['total_count']

In [ ]:
df_summary_simp = df_summary[['year_month', 'revenue', 'cogs', 'margin_$', 'margin_%', 'total_count_incoming', 'total_count', 'coverage']]

In [ ]:
df_summary_simp

In [ ]:
df_summary_simp.to_excel('./' + license_number + '_analysis.xlsx')

In [ ]:
df_summary_simp.to_excel('./verts_analysis.xlsx')

## Inventory

In [22]:
df_inventory_incoming = pd.merge(df_inventory, df_avg_incoming_price, on='package_id', how='left')
df_inventory_incoming.replace([np.inf], np.nan, inplace=True)
df_inv_null = df_inventory_incoming[df_inventory_incoming['per_unit_incoming'].isnull()]
df_inv_product = pd.merge(df_inv_null, df_avg_product, on='product_name', how='left')
df_inv_product.replace([np.inf], np.nan, inplace=True)
df_inv_product_price = df_inv_product[df_inv_product['per_unit_product'].notnull()]
df_inv_product_price['total_price'] = df_inv_product_price['quantity'] * df_inv_product_price['per_unit_product']

<ipython-input-22-3085e3cf9e2f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inv_product_price['total_price'] = df_inv_product_price['quantity'] * df_inv_product_price['per_unit_product']


In [23]:
inventory_product_value = df_inv_product_price['total_price'].sum()

In [24]:
df_inventory_incoming['total_price'] = df_inventory_incoming['quantity'] * df_inventory_incoming['per_unit_incoming']
inventory_value = df_inventory_incoming['total_price'].sum()

In [25]:
# inventory_product_value

In [26]:
total_inv_value = inventory_product_value + inventory_value


In [27]:
print(f'{total_inv_value:,.2f}')

184,148.99


In [28]:
inv_count_product = df_inv_product_price['per_unit_product'].count()
inv_count_incoming = df_inventory_incoming['per_unit_incoming'].count()
inv_count_total = df_inventory_incoming['quantity'].count()
inv_total_incoming = inv_count_product + inv_count_incoming

In [29]:
inv_count_total

353

In [30]:
# inv_count_total

In [31]:
inventory_coverage = inv_total_incoming / inv_count_total
print(f'{inventory_coverage:,.0%}')

88%


In [32]:
data = [['Date', today], 
        ['Value', total_inv_value], 
        ['Total Incoming', inv_total_incoming], 
        ['Total', inv_count_total], 
        ['Coverage', inventory_coverage],
        ['License', license_number],
        ['Legal Name', legal_name]]

In [33]:
df_inventory_license = pd.DataFrame(data, columns = ['0', '1'])

In [34]:
df_inventory_license

,0,1
0,Date,2022-02-12
1,Value,"184,148.99"
2,Total Incoming,311
3,Total,353
4,Coverage,0.88
5,License,MR283369
6,Legal Name,"Diem Lynn, LLC"


In [ ]:
#df_inventory_license.to_csv('../analysis/surveillance/'+COMPANY_IDENTIFIER+'/INV/' + COMPANY_IDENTIFIER + '_'+ str(today) + '_' +license_number +'_inventory_val.csv')




In [ ]:
#df_inventory_license.to_excel('./' + license_number + '_inventory_name.xlsx')

In [ ]:
#df_inventory_license.to_csv('dw_2022_01_inventory.csv')

# Notes

In [ ]:
# df_cogs[df_cogs['per_unit_incoming'].isnull() & df_cogs['per_unit_product'].notnull()]
# df_sales = df_sales[df_sales['license_number'] == 'C10-0000824-LIC']
# df_inventory = df_inventory[df_inventory['license_number'] == 'C10-0000824-LIC']
# df_sales['tx_total_price'].count()

In [ ]:
# %autoreload 2

# sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts/analysis")))
# sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

# from util import active_inventory_util as util

In [ ]:
# d = util.Download()
# d.download_dataframes(
#     incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
#     outgoing_transfer_packages_dataframe=company_outgoing_transfer_packages_dataframe,
#     sales_transactions_dataframe=company_sales_transactions_dataframe,
# )

In [ ]:
# q = util.Query()
# q.inventory_dates = INVENTORY_DATES
# q.company_name = COMPANY_NAME

# id_to_history = util.get_histories(d)
# util.print_counts(id_to_history)
# util.create_inventory_xlsx(id_to_history, q)

In [ ]:
# computed_inventory_package_records = util.create_inventory_dataframe_by_date(id_to_history, INVENTORY_DATE)
# computed_inventory_packages_dataframe = pandas.DataFrame(
#     computed_inventory_package_records,
#     columns=[
#         'package_id',
#         'Arrived Date',
#         'Product Category',
#         'Product Name',
#         'Current Quantity',
#         'Sold Date',
#     ]
# )
# computed_inventory_packages_dataframe

In [ ]:
# date_to_inventory_dataframe = {}
# for date, inventory_records in date_to_inventory_records.items():
#     date_to_inventory_dataframe[date] = pandas.DataFrame(
#         inventory_records,
#         columns=[
#             'package_id',
#             'Arrived Date',
#             'Product Category',
#             'Product Name',
#             'Current Quantity',
#             'Sold Date',
#         ]
#     )
    
# date_to_inventory_dataframe[list(date_to_inventory_dataframe.keys())[0]]

In [ ]:
# raw_incoming_transfer_packages_dataframe = pandas.read_excel('data/20210930/royal_apothecary_incoming_transfer_packages_20200101_20210930.xlsx', header=0)
# len(raw_incoming_transfer_packages_dataframe.index), raw_incoming_transfer_packages_dataframe.columns

In [ ]:
# for date, inventory_dataframe in date_to_inventory_dataframe.items():
#     print(date)
#     print(f'# of packages in inventory: {len(inventory_dataframe.index)}')

#     inventory_with_incoming_transfer_packages_dataframe = inventory_dataframe.astype({'package_id': 'int64'}).merge(incoming_transfer_packages_dataframe, on='package_id', how='inner', suffixes=('_l', '_r'))
# #     print(f'# of packages in inventory with incoming package: {len(inventory_with_incoming_transfer_packages_dataframe.index)}')
    
#     inventory_with_cost_records = inventory_with_incoming_transfer_packages_dataframe.to_dict('record')

#     total_valuation_cost = 0
#     import math
#     for inventory_with_cost_record in inventory_with_cost_records:
#         incoming_shipped_price = inventory_with_cost_record['shipper_wholesale_price']
#         if math.isnan(incoming_shipped_price):
#             incoming_shipped_price = 0
#         incoming_shipped_quantity = inventory_with_cost_record['shipped_quantity']
#         current_quantity = inventory_with_cost_record['Current Quantity']
#         total_valuation_cost += float(current_quantity) * (incoming_shipped_price / incoming_shipped_quantity)

#     print(f'Inventory valuation (based on COST): ${total_valuation_cost}')
#     print('')